<a href="https://colab.research.google.com/github/vilsonrodrigues/PortfolioDataScience/blob/master/Challange_ITA/Challange_ITA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Challange ITA 2021**

## 1 Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## 2 Data Understanding

### 2.0 Reading cvs files

Modelo de submissão da resposta ao server do ITA

In [ ]:
submission = pd.read_csv('http://www.comp.ita.br/~verri/dsc/edicoes/2021/submission-sample.csv')
submission.head(2)

,cent_price_cor,cent_trans_cor
0,0,0
1,0,0


In [ ]:
df_train = pd.read_csv('http://www.comp.ita.br/~verri/dsc/edicoes/2021/public-train.csv')
df_train.head(2)

,n,p,f,x,y,z,a1,a2,a3,a4,b1,b2,b3,b4,c1,c2,c3,c4,g1,g2,l1,l2,l3,l4,e1,e2,cent_price_cor,cent_trans_cor
0,709,0.7,0.2,16,12,7,3.8,0.24,2.3,0.28,1.4,0.2,3.1,0.17,4.7,0.19,2.2,0.21,0.186,0.5,0.147,1.5,0.089,1.6,1.5,2.6,-0.169,0.375
1,921,0.6,0.3,19,17,10,3.0,0.12,2.4,0.19,2.0,0.1,2.3,0.27,3.9,0.12,4.6,0.24,0.079,1.2,0.186,1.6,0.076,1.8,0.6,0.4,-0.075,0.234


In [ ]:
df_test = pd.read_csv('http://www.comp.ita.br/~verri/dsc/edicoes/2021/public-test.csv')
df_test.head(2)

,id,n,p,f,x,y,z,a1,a2,a3,a4,b1,b2,b3,b4,c1,c2,c3,c4,g1,g2,l1,l2,l3,l4,e1,e2
0,1,558,0.5,0.2,28,21,2,1.0,0.29,1.3,0.21,3.7,0.25,2.5,0.21,4.2,0.30,3.1,0.27,0.198,1.8,0.149,2.0,0.078,1.7,0.5,2.9
1,2,910,0.9,0.3,28,19,9,2.2,0.14,1.6,0.14,4.7,0.12,4.9,0.11,4.9,0.22,2.2,0.16,0.172,1.0,0.086,0.8,0.030,0.8,0.2,1.6


###  2.1 Data Description

**Features**

* **n**: Indicador do número de traders/players no sistema. São os consumidores que estão interessados naquele espaço aéreo. Eles podem adquirir ou vendê-los, fazendo uso de especulação.

* **p**: Fração de traders.

* **f**: Indicador do interesse desses traders/players em relação as permissões.

* **x**, **y**, **z**: Dimensões do espaço aéreo.

* **a1** e **a2**: Média e desvio padrão do coeficiente do preço fundamental dos consumidores.

* **a3** e **a4**: O mesmo para os traders.
 
* **b1** e **b2**: Média e desvio padrão do coeficiente do preço de mercado dos consumidores.

* **b3** e **b4**: O mesmo para os traders.

* **c1**, **c2**: Média e desvio padrão do coeficiente do preço aleatorio dos consumidores.

* **c3** e **c4**: O mesmo para os traders.
 
* **g1** e **g2**: Média e desvio padrão do grau de agressividade dos consumidores

 * **l1**, **l2**: Média e dp do coeficiente de desvalorização para os consumidores

 * **l3** e **l4**: O mesmo para os traders.

* **e1** e **e2**: Variabilidade no preço fundamental dos consumidores e traders; respectivamente


**Targets**

* **cent_price_cor**: Correlação entre preço final e centralidade das permissões de vôo.

* **cent_trans_cor**: Idem para o número de transações.

**A centralidade é medida como o número de sequências (caminhos) mínimos que a permissão pertence.**

**A correlação usada foi a de person**

### 2.2 Data Quality

Sem dados faltantes. Sem nenhum atributo categórico

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11940 entries, 0 to 11939
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   n               11940 non-null  int64  
 1   p               11940 non-null  float64
 2   f               11940 non-null  float64
 3   x               11940 non-null  int64  
 4   y               11940 non-null  int64  
 5   z               11940 non-null  int64  
 6   a1              11940 non-null  float64
 7   a2              11940 non-null  float64
 8   a3              11940 non-null  float64
 9   a4              11940 non-null  float64
 10  b1              11940 non-null  float64
 11  b2              11940 non-null  float64
 12  b3              11940 non-null  float64
 13  b4              11940 non-null  float64
 14  c1              11940 non-null  float64
 15  c2              11940 non-null  float64
 16  c3              11940 non-null  float64
 17  c4              11940 non-null 

O mesmo ocorre nos dados de teste

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8059 entries, 0 to 8058
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      8059 non-null   int64  
 1   n       8059 non-null   int64  
 2   p       8059 non-null   float64
 3   f       8059 non-null   float64
 4   x       8059 non-null   int64  
 5   y       8059 non-null   int64  
 6   z       8059 non-null   int64  
 7   a1      8059 non-null   float64
 8   a2      8059 non-null   float64
 9   a3      8059 non-null   float64
 10  a4      8059 non-null   float64
 11  b1      8059 non-null   float64
 12  b2      8059 non-null   float64
 13  b3      8059 non-null   float64
 14  b4      8059 non-null   float64
 15  c1      8059 non-null   float64
 16  c2      8059 non-null   float64
 17  c3      8059 non-null   float64
 18  c4      8059 non-null   float64
 19  g1      8059 non-null   float64
 20  g2      8059 non-null   float64
 21  l1      8059 non-null   float64
 22  

In [ ]:
df_train.select_dtypes(include=['int64','float64']).describe()

,n,p,f,x,y,z,a1,a2,a3,a4,b1,b2,b3,b4,c1,c2,c3,c4,g1,g2,l1,l2,l3,l4,e1,e2,cent_price_cor,cent_trans_cor
count,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000,11940.000000
mean,548.845059,0.700687,0.350176,19.929564,19.972697,5.458124,3.008049,0.200654,2.989573,0.199744,3.006767,0.200666,2.993501,0.201355,3.005938,0.199647,3.013300,0.200454,0.100422,1.255310,0.100020,1.252814,0.049928,1.252152,1.547931,1.552647,-0.202591,0.362586
std,258.342510,0.122642,0.095939,6.077175,6.089534,2.874370,1.156649,0.057774,1.153929,0.058152,1.145294,0.057999,1.153200,0.058083,1.155266,0.058393,1.162371,0.058065,0.057729,0.435027,0.058071,0.435154,0.028666,0.434480,0.839320,0.845237,0.116617,0.115221
min,100.000000,0.500000,0.200000,10.000000,10.000000,1.000000,1.000000,0.100000,1.000000,0.100000,1.000000,0.100000,1.000000,0.100000,1.000000,0.100000,1.000000,0.100000,0.000000,0.500000,0.000000,0.500000,0.000000,0.500000,0.100000,0.100000,-0.613000,-0.399000
25%,322.000000,0.600000,0.300000,15.000000,15.000000,3.000000,2.000000,0.150000,2.000000,0.150000,2.000000,0.150000,2.000000,0.150000,2.000000,0.150000,2.000000,0.150000,0.051000,0.900000,0.049000,0.900000,0.025000,0.900000,0.800000,0.800000,-0.284000,0.294000
50%,551.000000,0.700000,0.400000,20.000000,20.000000,5.000000,3.000000,0.200000,3.000000,0.200000,3.000000,0.200000,3.000000,0.200000,3.000000,0.200000,3.000000,0.200000,0.100000,1.300000,0.101000,1.300000,0.050000,1.300000,1.500000,1.600000,-0.205000,0.372000
75%,770.000000,0.800000,0.400000,25.000000,25.000000,8.000000,4.000000,0.250000,4.000000,0.250000,4.000000,0.250000,4.000000,0.250000,4.000000,0.250000,4.000000,0.250000,0.151000,1.600000,0.151000,1.600000,0.075000,1.600000,2.300000,2.300000,-0.122000,0.443000
max,1000.000000,0.900000,0.500000,30.000000,30.000000,10.000000,5.000000,0.300000,5.000000,0.300000,5.000000,0.300000,5.000000,0.300000,5.000000,0.300000,5.000000,0.300000,0.200000,2.000000,0.200000,2.000000,0.100000,2.000000,3.000000,3.000000,0.266000,0.717000


In [ ]:
df_test.select_dtypes(include=['int64','float64']).describe()

,id,n,p,f,x,y,z,a1,a2,a3,a4,b1,b2,b3,b4,c1,c2,c3,c4,g1,g2,l1,l2,l3,l4,e1,e2
count,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000,8059.000000
mean,4030.000000,549.207842,0.700000,0.351334,20.016255,20.017868,5.489267,3.006576,0.200403,2.973185,0.199985,3.001675,0.199290,2.978769,0.200613,3.018613,0.200216,2.966100,0.199765,0.099895,1.248244,0.100925,1.246495,0.049729,1.249659,1.545812,1.556744
std,2326.577243,258.410608,0.121707,0.095852,6.075116,6.028861,2.860404,1.139120,0.057958,1.152527,0.057480,1.160048,0.058117,1.150934,0.057144,1.162174,0.057786,1.151742,0.058150,0.057641,0.436101,0.058157,0.432854,0.029121,0.431852,0.838634,0.835856
min,1.000000,100.000000,0.500000,0.200000,10.000000,10.000000,1.000000,1.000000,0.100000,1.000000,0.100000,1.000000,0.100000,1.000000,0.100000,1.000000,0.100000,1.000000,0.100000,0.000000,0.500000,0.000000,0.500000,0.000000,0.500000,0.100000,0.100000
25%,2015.500000,326.000000,0.600000,0.300000,15.000000,15.000000,3.000000,2.000000,0.150000,2.000000,0.150000,2.000000,0.150000,2.000000,0.150000,2.000000,0.150000,2.000000,0.150000,0.050000,0.900000,0.050000,0.900000,0.024000,0.900000,0.800000,0.800000
50%,4030.000000,551.000000,0.700000,0.400000,20.000000,20.000000,5.000000,3.000000,0.200000,3.000000,0.200000,3.000000,0.200000,3.000000,0.200000,3.000000,0.200000,3.000000,0.200000,0.099000,1.200000,0.102000,1.200000,0.050000,1.200000,1.500000,1.600000
75%,6044.500000,772.000000,0.800000,0.400000,25.000000,25.000000,8.000000,4.000000,0.250000,4.000000,0.250000,4.000000,0.250000,4.000000,0.250000,4.000000,0.250000,3.900000,0.250000,0.150000,1.600000,0.152000,1.600000,0.075000,1.600000,2.300000,2.300000
max,8059.000000,1000.000000,0.900000,0.500000,30.000000,30.000000,10.000000,5.000000,0.300000,5.000000,0.300000,5.000000,0.300000,5.000000,0.300000,5.000000,0.300000,5.000000,0.300000,0.200000,2.000000,0.200000,2.000000,0.100000,2.000000,3.000000,3.000000


### 2.3 Data Exploration

In [ ]:
df_train.head()

,n,p,f,x,y,z,a1,a2,a3,a4,b1,b2,b3,b4,c1,c2,c3,c4,g1,g2,l1,l2,l3,l4,e1,e2,cent_price_cor,cent_trans_cor
0,709,0.7,0.2,16,12,7,3.8,0.24,2.3,0.28,1.4,0.20,3.1,0.17,4.7,0.19,2.2,0.21,0.186,0.5,0.147,1.5,0.089,1.6,1.5,2.6,-0.169,0.375
1,921,0.6,0.3,19,17,10,3.0,0.12,2.4,0.19,2.0,0.10,2.3,0.27,3.9,0.12,4.6,0.24,0.079,1.2,0.186,1.6,0.076,1.8,0.6,0.4,-0.075,0.234
2,177,0.8,0.4,14,12,5,3.2,0.17,1.8,0.18,2.5,0.24,1.8,0.28,4.4,0.24,3.9,0.28,0.036,1.4,0.048,0.7,0.073,0.6,2.7,0.3,-0.177,0.389
3,415,0.7,0.5,24,11,2,1.3,0.17,1.5,0.18,3.1,0.16,3.6,0.16,2.6,0.11,4.9,0.20,0.063,1.1,0.151,0.8,0.022,1.1,0.5,0.1,-0.102,0.358
4,802,0.8,0.4,21,10,3,4.4,0.15,2.6,0.13,2.2,0.28,1.4,0.26,4.3,0.24,2.4,0.24,0.044,1.9,0.123,1.9,0.046,1.1,2.0,0.6,-0.034,0.180


Correlação. Os atributos tem quase nenhuma correlação entre si, apenas com os targets

In [ ]:
corr = df_train.corr()
corr = corr.style.background_gradient(cmap='Blues')
corr

,n,p,f,x,y,z,a1,a2,a3,a4,b1,b2,b3,b4,c1,c2,c3,c4,g1,g2,l1,l2,l3,l4,e1,e2,cent_price_cor,cent_trans_cor
n,1.000000,0.011894,0.002752,-0.002173,-0.006470,0.015184,0.008776,-0.011888,-0.004741,-0.006187,-0.010738,-0.008082,0.005773,-0.009081,0.013390,-0.012974,0.003213,-0.003228,-0.007706,0.002346,0.003786,-0.002720,-0.012730,0.006319,0.004058,-0.003441,0.006058,0.003236
p,0.011894,1.000000,-0.001149,0.000694,-0.006581,-0.011965,-0.002200,-0.012795,0.005200,-0.023065,0.000313,0.002444,0.018077,0.008147,0.003944,0.012279,-0.000634,0.007425,0.001459,0.004579,0.008466,0.011060,-0.005461,0.007454,-0.003013,0.014696,-0.001929,0.002209
f,0.002752,-0.001149,1.000000,0.007154,-0.006816,-0.005638,-0.007300,0.015990,0.011165,-0.010366,-0.010218,0.002907,-0.006773,-0.005153,0.005481,0.000519,0.001271,0.003639,0.012050,-0.006153,-0.011456,0.003549,0.021002,0.003146,0.014161,0.000960,0.006039,-0.009006
x,-0.002173,0.000694,0.007154,1.000000,0.001947,-0.000051,-0.004799,-0.007603,0.011545,0.008536,0.001048,0.003018,-0.001265,0.015870,-0.009971,0.000081,0.008812,-0.008044,0.000886,0.010167,-0.007404,-0.008358,0.006677,-0.013591,-0.011036,-0.018221,0.004589,0.001371
y,-0.006470,-0.006581,-0.006816,0.001947,1.000000,0.009740,0.007585,0.003267,-0.006798,0.008209,0.009634,0.008333,-0.002140,-0.003777,-0.000406,0.006460,-0.010500,0.005715,-0.005759,-0.014660,0.000410,-0.007314,-0.007359,-0.003853,-0.028300,-0.004679,0.009405,0.002117
z,0.015184,-0.011965,-0.005638,-0.000051,0.009740,1.000000,0.014624,-0.001820,0.007046,0.001685,-0.017930,0.006164,-0.009432,0.002773,0.003148,-0.011446,-0.004383,-0.003384,0.004032,-0.001704,-0.012491,0.012396,-0.006203,-0.015276,-0.007582,-0.008284,0.003054,-0.005470
a1,0.008776,-0.002200,-0.007300,-0.004799,0.007585,0.014624,1.000000,-0.008066,0.001901,0.001006,0.010554,0.006299,0.003041,-0.004212,0.010163,-0.003725,0.011747,0.003026,-0.004382,-0.009151,0.006216,0.008889,-0.009120,-0.005975,0.001682,-0.002644,-0.006230,0.000264
a2,-0.011888,-0.012795,0.015990,-0.007603,0.003267,-0.001820,-0.008066,1.000000,0.002934,0.007751,-0.012390,0.008821,0.014740,-0.004750,0.000919,0.000548,0.008266,0.005347,-0.004813,-0.006165,-0.010240,-0.003133,0.010833,-0.010692,0.013576,0.003067,0.000586,-0.000042
a3,-0.004741,0.005200,0.011165,0.011545,-0.006798,0.007046,0.001901,0.002934,1.000000,0.006604,0.022241,-0.000384,0.009517,-0.017109,-0.019089,-0.003205,0.000638,-0.010375,0.002244,-0.003737,0.002333,-0.015459,-0.001076,-0.002303,0.007071,-0.015244,-0.014581,0.000141
a4,-0.006187,-0.023065,-0.010366,0.008536,0.008209,0.001685,0.001006,0.007751,0.006604,1.000000,0.002058,-0.006771,-0.016550,0.006977,0.000207,-0.006262,-0.002123,-0.007631,0.016740,-0.014895,-0.001353,0.011839,0.006501,0.004951,-0.007423,0.008875,-0.000458,-0.007882


In [ ]:
import plotly.express as px

Usando as distribuições da pra ver que os dados estão em torno da média, que é a situação ideal

In [ ]:
px.histogram(df_train, x = 'cent_price_cor')

In [ ]:
px.violin(df_train, y='cent_price_cor', box = True, points = 'all')

Os dados mostram que a distribuição está em torno da média mas tem alguns outliers na calda da distribuição

In [ ]:
px.histogram(df_train, x = 'cent_trans_cor')

In [ ]:
px.violin(df_train, y = 'cent_trans_cor', box = True, points = 'all')

Temos classes balenceadas no número de traders. Provavelmente o outlier anterior é esse número de players igual a 1000

In [ ]:
px.histogram(df_train, x = 'n')

In [ ]:
px.histogram(df_train, x = 'xyz')

In [ ]:
px.violin(df_train, y='xyz', box = True, points = 'all')

In [ ]:
df_train['xyz'] = np.log(df_train['xyz'])

In [ ]:
px.histogram(df_train, x = 'xyz')

In [ ]:
px.violin(df_train, y='xyz', box = True, points = 'all')

In [ ]:
px.histogram(df_train, x = 'cent_trans_cor')
px.violin(df_train, y='cent_price_cor', box = True, points = 'all')

In [ ]:
df_train.head()

,n,p,f,x,y,z,a1,a2,a3,a4,b1,b2,b3,b4,c1,c2,c3,c4,g1,g2,l1,l2,l3,l4,e1,e2,cent_price_cor,cent_trans_cor,xyz
0,709,0.7,0.2,16,12,7,3.8,0.24,2.3,0.28,1.4,0.20,3.1,0.17,4.7,0.19,2.2,0.21,0.186,0.5,0.147,1.5,0.089,1.6,1.5,2.6,-0.169,0.375,7.203406
1,921,0.6,0.3,19,17,10,3.0,0.12,2.4,0.19,2.0,0.10,2.3,0.27,3.9,0.12,4.6,0.24,0.079,1.2,0.186,1.6,0.076,1.8,0.6,0.4,-0.075,0.234,8.080237
2,177,0.8,0.4,14,12,5,3.2,0.17,1.8,0.18,2.5,0.24,1.8,0.28,4.4,0.24,3.9,0.28,0.036,1.4,0.048,0.7,0.073,0.6,2.7,0.3,-0.177,0.389,6.733402
3,415,0.7,0.5,24,11,2,1.3,0.17,1.5,0.18,3.1,0.16,3.6,0.16,2.6,0.11,4.9,0.20,0.063,1.1,0.151,0.8,0.022,1.1,0.5,0.1,-0.102,0.358,6.269096
4,802,0.8,0.4,21,10,3,4.4,0.15,2.6,0.13,2.2,0.28,1.4,0.26,4.3,0.24,2.4,0.24,0.044,1.9,0.123,1.9,0.046,1.1,2.0,0.6,-0.034,0.180,6.445720



```
df = px.data.iris()
fig = px.parallel_coordinates(df, color="species_id", labels={"species_id": "Species",
                  "sepal_width": "Sepal Width", "sepal_length": "Sepal Length",
                  "petal_width": "Petal Width", "petal_length": "Petal Length", },
                    color_continuous_scale=px.colors.diverging.Tealrose, color_continuous_midpoint=2)
fig.show()

```



## 3 Data Ideas

df_train['b1/e1'] = df_train['b1'] / df_train['e1']

df_test['b1/e1'] = df_test['b1'] / df_test['e1']


---


df_train['(c1-c3)/c2'] = (df_train['c1'] - df_train['c3'])/df_train['c2']

df_test['(c1-c3)/c2'] = (df_test['c1'] - df_test['c3'])/df_test['c2']


---


df_train['(c1-c3)/c4'] = (df_train['c1'] - df_train['c3'])/df_train['c4']

df_test['(c1-c3)/c4'] = (df_test['c1'] - df_test['c3'])/df_test['c4']


---


df_train['(a1-b1)/b2'] = (df_train['a1'] - df_train['b1'])/df_train['b2']

df_test['(a1-b1)/b2'] = (df_test['a1'] - df_test['b1'])/df_test['b2']


---


df_train['(a1-b1)/a2'] = (df_train['a1'] - df_train['b1'])/df_train['a2']

df_test['(a1-b1)/a2'] = (df_test['a1'] - df_test['b1'])/df_test['a2']


---


df_train['g1/a1'] = df_train['g1']/df_train['a1'] 

df_test['g1/a1'] = df_test['g1']/df_test['a1'] 


---



df_train['(b1*a1)/c1'] = (df_train['b1'] * df_train['a1'])/df_train['c1']

df_test['(b1*a1)/c1'] = (df_test['b1'] * df_test['a1'])/df_test['c1']



---



df_train['c1*a1*b1'] = df_train['c1'] * df_train['a1'] * df_train['b1']

df_test['c1*a1*b1'] = df_test['c1'] * df_test['a1'] * df_test['b1']



---



df_train['c1/c2'] = df_train['c1'] / df_train['c2']

df_test['c1/c2'] = df_test['c1'] / df_test['c2']


---


df_train['c3/c4'] = df_train['c3'] / df_train['c4']

df_test['c3/c4'] = df_test['c3'] / df_test['c4']



---



df_train['b1-a1'] = df_train['b1'] - df_train['a1']

df_test['b1-a1'] = df_test['b1'] - df_test['a1']



---



df_train['(b1-a1)/b2'] = (df_train['b1'] - df_train['a1'])/df_train['b2']

df_test['(b1-a1)/b2'] = (df_test['b1'] - df_test['a1'])/df_test['b2']



---



df_train['(a1-b3)/b4'] = (df_train['a1'] - df_train['b3'])/df_train['b4']

df_test['(a1-b3)/b4'] = (df_test['a1'] - df_test['b3'])/df_test['b4']



---



df_train['l1*g1'] = df_train['l1'] * df_train['g1']

df_test['l1*g1'] = df_test['l1'] * df_test['g1']



---



df_train['l1*f'] = df_train['l1'] * df_train['f']

df_test['l1*f'] = df_test['l1'] * df_test['f']



---



df_train['a3*f*p'] = df_train['a3'] * df_train['f'] * df_train['p']

df_test['a3*f*p'] = df_test['a3'] * df_test['f'] * df_test['p']



---


df_train['a3*f'] = df_train['a3'] * df_train['f'] 

df_test['a3*f'] = df_test['a3'] * df_test['f'] 



---



df_train['b3*f'] = df_train['b3'] * df_train['f'] 

df_test['b3*f'] = df_test['b3'] * df_test['f'] 



---



df_train['(e1-e2)/f'] = (df_train['e1'] - df_train['e2']) * df_train['f']

df_test['(e1-e2)/f'] = (df_test['e1'] - df_test['e2']) * df_test['f']


---


df_train['(b3-c1)/c2'] = (df_train['b3'] - df_train['c1']) / df_train['c2']

df_test['(b3-c1)/c2'] = (df_test['b3'] - df_test['c1']) / df_test['c2']

## 4 Metrics

A métrica escolhida pelo desafio foi a MAE - Mean Absolute Error, ele calcula o erro entre o valor observado e o valor predito e depois faz a média depois de somar as diferenças, o erro médio absoluto usa a mesma escala dos dados que estão sendo medidos. Isso é conhecido como uma medida de precisão dependente da escala e, portanto, não pode ser usado para fazer comparações entre séries usando escalas diferentes.

O MSE - Mean Squared Error funciona de forma semelhante, medindo a média dos quadrados dos erros. O MSE é uma medida da qualidade de um estimador - é sempre não negativo e os valores mais próximos de zero são melhores. Por isso também levaremos em conta o MSE na hora de analisar os modelos.

![Alt text](http://econbrowser.com/wp-content/uploads/2019/07/msemae.png)


## 5 Modeling

In [ ]:
df_train.head(2)

,n,p,f,x,y,z,a1,a2,a3,a4,b1,b2,b3,b4,c1,c2,c3,c4,g1,g2,l1,l2,l3,l4,e1,e2,cent_price_cor,cent_trans_cor,aaa
0,709,0.7,0.2,16,12,7,3.8,0.24,2.3,0.28,1.4,0.2,3.1,0.17,4.7,0.19,2.2,0.21,0.186,0.5,0.147,1.5,0.089,1.6,1.5,2.6,-0.169,0.375,-0.22
1,921,0.6,0.3,19,17,10,3.0,0.12,2.4,0.19,2.0,0.1,2.3,0.27,3.9,0.12,4.6,0.24,0.079,1.2,0.186,1.6,0.076,1.8,0.6,0.4,-0.075,0.234,0.06


### 5.0 Utils

Achatando os dados para facilitar o treinamento usando normalização

In [ ]:
def normalize_dataset(df_train, df_test):
  #Instanciando o normalizador
  scaler = StandardScaler()
  #Treinando com os dados
  scaler.fit(df_train)
  #Normalizando as features
  df_train = scaler.transform(df_train)
  df_test  = scaler.transform(df_test)
  return  df_train, df_test

Separa o dataframe que temos os alvos em train e test. Fazemos o split pra as correlações de transação e preço em relação a centralidade

In [ ]:
def split_dataset(X, y):
  #Split, separa em dados de treino e teste, sendo 80% para treino
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
  return X_train, X_test, y_train, y_test

Constrói o modelo com 2 saídas para cada atributo a ser previsto

In [ ]:
def build_model(X):
  n_input = X.shape[1]
  #Batch Normalization está causando overfitting no modelo
  model = tf.keras.Sequential([
                            tf.keras.Input(shape=(n_input,)),
                            tf.keras.layers.Dense(n_input, kernel_initializer = tf.keras.initializers.HeNormal(), kernel_regularizer = tf.keras.regularizers.l2(l=0.01)),                                                                                    
                            tf.keras.layers.Dropout(0.25),                                                  
                            #tf.keras.layers.Activation(tf.nn.relu),
                            tf.keras.layers.Dense(2)
  ])  
  #optimizer = tf.keras.optimizers.RMSprop(0.001)
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(loss = 'mae',
                  optimizer = optimizer,
                  metrics = ['mae', 'mse'])
  return model

Função de treino, feita para avaliação

In [ ]:
def train(X_train, X_test, y_train, y_test, epochs = 30):
  model = build_model(X_train)
  history = model.fit(x = X_train,
                      y = y_train,
                      batch_size = 32,
                      epochs = epochs,
                      validation_data = (X_test, y_test),            
                      verbose = 1)
  return model

Função de treino pra treinar com todos os dados do desafio

In [ ]:
def final_train(X, y, epochs = 30):
  model = build_model(X)
  history = model.fit(x = X,
                      y = y,
                      batch_size = 32,
                      epochs = epochs,
                      verbose = 1,
                      callbacks = [tf.keras.callbacks.EarlyStopping(monitor='mae', mode='auto', patience = 25)])
  return model

### 5.1 Applying normalization

In [ ]:
df_train.head()

,a1,a3,b1,b3,c1,c3,c4,g1,g2,l1,l3,e1,e2,cent_price_cor,cent_trans_cor
0,3.8,2.3,1.4,3.1,4.7,2.2,0.21,0.186,0.5,0.147,0.089,1.5,2.6,-0.169,0.375
1,3.0,2.4,2.0,2.3,3.9,4.6,0.24,0.079,1.2,0.186,0.076,0.6,0.4,-0.075,0.234
2,3.2,1.8,2.5,1.8,4.4,3.9,0.28,0.036,1.4,0.048,0.073,2.7,0.3,-0.177,0.389
3,1.3,1.5,3.1,3.6,2.6,4.9,0.20,0.063,1.1,0.151,0.022,0.5,0.1,-0.102,0.358
4,4.4,2.6,2.2,1.4,4.3,2.4,0.24,0.044,1.9,0.123,0.046,2.0,0.6,-0.034,0.180


In [ ]:
df_test.head(2)

,id,a1,a3,b1,b3,c1,c3,c4,g1,g2,l1,l3,e1,e2
0,1,1.0,1.3,3.7,2.5,4.2,3.1,0.27,0.198,1.8,0.149,0.078,0.5,2.9
1,2,2.2,1.6,4.7,4.9,4.9,2.2,0.16,0.172,1.0,0.086,0.030,0.2,1.6


Aplicando normalização

In [ ]:
data_train, data_test = normalize_dataset(df_train.drop(columns=['cent_price_cor','cent_trans_cor']), df_test.drop(columns=['id']))

### 5.2 Initial Tests

Separando os dados em treino e teste

In [ ]:
X_train, X_test, y_train, y_test = split_dataset(data_train, df_train[['cent_price_cor','cent_trans_cor']])

In [ ]:
model = train(X_train, X_test, y_train, y_test)

Epoch 1/30
299/299 [==============================] - 1s 3ms/step - loss: 1.7447 - mae: 0.8137 - mse: 1.1463 - val_loss: 0.6258 - val_mae: 0.1817 - val_mse: 0.0521
Epoch 2/30
299/299 [==============================] - 1s 2ms/step - loss: 0.5425 - mae: 0.2140 - mse: 0.0772 - val_loss: 0.1736 - val_mae: 0.0955 - val_mse: 0.0144
Epoch 3/30
299/299 [==============================] - 1s 2ms/step - loss: 0.1460 - mae: 0.0983 - mse: 0.0155 - val_loss: 0.0977 - val_mae: 0.0912 - val_mse: 0.0132
Epoch 4/30
299/299 [==============================] - 1s 2ms/step - loss: 0.0975 - mae: 0.0932 - mse: 0.0138 - val_loss: 0.0913 - val_mae: 0.0902 - val_mse: 0.0130
Epoch 5/30
299/299 [==============================] - 1s 2ms/step - loss: 0.0938 - mae: 0.0929 - mse: 0.0138 - val_loss: 0.0908 - val_mae: 0.0904 - val_mse: 0.0130
Epoch 6/30
299/299 [==============================] - 1s 2ms/step - loss: 0.0924 - mae: 0.0920 - mse: 0.0135 - val_loss: 0.0905 - val_mae: 0.0902 - val_mse: 0.0129
Epoch 7/30
299/2

### 5.3 Searching for the best initial architecture

A combinação de uma camada oculta com o número de nodes igual ao de entradas e um dropout de 0.25 construiu o modelo de melhor resultado da competição até o presente momento (07/05/21)

In [ ]:
model = final_train(data_train, df_train[['cent_price_cor','cent_trans_cor']])

Realizando Predição

In [ ]:
results = model.predict(data_test)

Salvando o resultado em dataframe no formato exigido pela competição

In [ ]:
predictions = pd.DataFrame(results, columns=['cent_price_cor', 'cent_trans_cor'])
predictions

Exibindo as 20 primeiras linhas da predição

In [ ]:
predictions.head(20)

Persistindo em disco

In [ ]:
predictions.to_csv('predictions.csv')

Essa combinação simples produziu um score de 0.18429 

### 5.4 Using Wandb

O WandB permite acompanhar o progresso do treinamento, salvar os melhores modelos de cada época, entre outras funcionalidades

Com a melhor arquitetura definida, foi levantada várias combinações de features para melhorar o desempenho do modelo, iremos utilizar o wandb para verificar quais delas influenciam na diminuição do erro MAE

Instalando e importando o WandB

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

Logando na conta

In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Declarando a entidade

In [ ]:
entity = "vilsonrodrigues07"

Nome do projeto

In [ ]:
name_project = "challange_ita_2021"

Instanciando os parâmetros do hyper tuning sweep

In [ ]:
sweep_config = {
                'method': 'grid', 
                'metric': {
                  'name': 'val_mae',
                  'goal': 'maximize'   
                },
                'parameters': {
                  'new_feature': {
                    'values': ['b1/e1','(c1-c3)/c2','(c1-c3)/c4','(a1-b1)/b2','(a1-b1)/a2','g1/a1',
                               '(b1*a1)/c1','c1*a1*b1','c1/c2','c3/c4','b1-a1','(b1-a1)/b2','(a1-b3)/b4',
                               'l1*g1','l1*f','a3*f*p','a3*f','b3*f','(e1-e2)/f','(b3-c1)/c2']
                  },
                  'num_epochs': {
                      'values': [30,50,70]
                  }
                }
            }

Nova função de treino

In [ ]:
def train_wandb():

  #Configurações padrões
  defaults = dict(new_feature = 'b1/e1',
                  num_epochs = 50)
  
  #Instanciando um run
  run = wandb.init(project = name_project, config = defaults)

  #Importando as variáveis de configuração
  config = wandb.config

  #Todas as combinações de features pensadas pela equipe
  if config.new_feature == 'b1/e1':
    df_train['b1/e1'] = df_train['b1'] / df_train['e1']
    df_test['b1/e1'] = df_test['b1'] / df_test['e1']

  if config.new_feature == '(c1-c3)/c2':
    df_train['(c1-c3)/c2'] = (df_train['c1'] - df_train['c3'])/df_train['c2']
    df_test['(c1-c3)/c2'] = (df_test['c1'] - df_test['c3'])/df_test['c2']

  if config.new_feature == '(c1-c3)/c4':
    df_train['(c1-c3)/c4'] = (df_train['c1'] - df_train['c3'])/df_train['c4']
    df_test['(c1-c3)/c4'] = (df_test['c1'] - df_test['c3'])/df_test['c4']

  if config.new_feature == '(a1-b1)/b2':
    df_train['(a1-b1)/b2'] = (df_train['a1'] - df_train['b1'])/df_train['b2']
    df_test['(a1-b1)/b2'] = (df_test['a1'] - df_test['b1'])/df_test['b2']

  if config.new_feature == '(a1-b1)/a2':
    df_train['(a1-b1)/a2'] = (df_train['a1'] - df_train['b1'])/df_train['a2']
    df_test['(a1-b1)/a2'] = (df_test['a1'] - df_test['b1'])/df_test['a2']

  if config.new_feature == 'g1/a1':
    df_train['g1/a1'] = df_train['g1']/df_train['a1'] 
    df_test['g1/a1'] = df_test['g1']/df_test['a1'] 

  if config.new_feature == '(b1*a1)/c1':
    df_train['(b1*a1)/c1'] = (df_train['b1'] * df_train['a1'])/df_train['c1']
    df_test['(b1*a1)/c1'] = (df_test['b1'] * df_test['a1'])/df_test['c1']

  if config.new_feature == 'c1*a1*b1':
    df_train['c1*a1*b1'] = df_train['c1'] * df_train['a1'] * df_train['b1']
    df_test['c1*a1*b1'] = df_test['c1'] * df_test['a1'] * df_test['b1']

  if config.new_feature == 'c1/c2':
    df_train['c1/c2'] = df_train['c1'] / df_train['c2']
    df_test['c1/c2'] = df_test['c1'] / df_test['c2']

  if config.new_feature == 'c3/c4':
    df_train['c3/c4'] = df_train['c3'] / df_train['c4']
    df_test['c3/c4'] = df_test['c3'] / df_test['c4']

  if config.new_feature == 'b1-a1':
    df_train['b1-a1'] = df_train['b1'] - df_train['a1']
    df_test['b1-a1'] = df_test['b1'] - df_test['a1']

  if config.new_feature == '(b1-a1)/b2':
    df_train['(b1-a1)/b2'] = (df_train['b1'] - df_train['a1'])/df_train['b2']
    df_test['(b1-a1)/b2'] = (df_test['b1'] - df_test['a1'])/df_test['b2']

  if config.new_feature == '(a1-b3)/b4':
    df_train['(a1-b3)/b4'] = (df_train['a1'] - df_train['b3'])/df_train['b4']
    df_test['(a1-b3)/b4'] = (df_test['a1'] - df_test['b3'])/df_test['b4']

  if config.new_feature == 'l1*g1':
    df_train['l1*g1'] = df_train['l1'] * df_train['g1']
    df_test['l1*g1'] = df_test['l1'] * df_test['g1']

  if config.new_feature == 'l1*f':
    df_train['l1*f'] = df_train['l1'] * df_train['f']
    df_test['l1*f'] = df_test['l1'] * df_test['f']

  if config.new_feature == 'a3*f*p':
    df_train['a3*f*p'] = df_train['a3'] * df_train['f'] * df_train['p']
    df_test['a3*f*p'] = df_test['a3'] * df_test['f'] * df_test['p']

  if config.new_feature == 'a3*f':
    df_train['a3*f'] = df_train['a3'] * df_train['f'] 
    df_test['a3*f'] = df_test['a3'] * df_test['f'] 

  if config.new_feature == 'b3*f':
    df_train['b3*f'] = df_train['b3'] * df_train['f'] 
    df_test['b3*f'] = df_test['b3'] * df_test['f'] 

  if config.new_feature == '(e1-e2)/f':
    df_train['(e1-e2)/f'] = (df_train['e1'] - df_train['e2']) * df_train['f']
    df_test['(e1-e2)/f'] = (df_test['e1'] - df_test['e2']) * df_test['f']

  if config.new_feature == '(b3-c1)/c2':
    df_train['(b3-c1)/c2'] = (df_train['b3'] - df_train['c1']) / df_train['c2']
    df_test['(b3-c1)/c2'] = (df_test['b3'] - df_test['c1']) / df_test['c2']

  #Aplicando a normalização nas features
  data_train, _ = normalize_dataset(df_train.drop(columns=['cent_price_cor','cent_trans_cor']), df_test.drop(columns=['id']))
  #Carregando arquitetura 
  model = build_model(data_train)
  history = model.fit(x = data_train,
                      y = df_train[['cent_price_cor','cent_trans_cor']],
                      batch_size = 32,
                      epochs = config.num_epochs,                      
                      callbacks = [WandbCallback()],           
                      verbose = 1)

Gerando um sweep id

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity = entity, project = name_project)

Create sweep with ID: zv1fnykz
Sweep URL: https://wandb.ai/vilsonrodrigues07/challange_ita_2021/sweeps/zv1fnykz


Iniciando treinamento, ele vai realizar 40 combinações diferentes

In [ ]:
wandb.agent(sweep_id = sweep_id, function = train_wandb, count = 40)

No wandb selecionamos as 5 melhores features que minimizaram o MAE e a Loss

**Best loss**

(a1-b1)/b2

(b1*a1)/c1

g1/a1

c1/c2

c3/c4

**Best MAE**

(a1-b3)/b4

(b1*a1)/c1

(c1-c3)/c2

(b1-a1)/b2

(c1-c3)/c4

(c1-c3)/c4

b1/e1

### 5.5 Final Model

Agora com os melhores parâmetros achados pelo sweeps, vamos construir o modelo final

Inserindo as features

In [ ]:
df_train['(c1-c3)/c4'] = (df_train['c1'] - df_train['c3'])/df_train['c4']
df_test['(c1-c3)/c4'] = (df_test['c1'] - df_test['c3'])/df_test['c4']
df_train['g1/a1'] = df_train['g1']/df_train['a1']
df_test['g1/a1'] = df_test['g1']/df_test['a1']
df_train['(a1-b3)/b4'] = (df_train['a1'] - df_train['b3'])/df_train['b4']
df_test['(a1-b3)/b4'] = (df_test['a1'] - df_test['b3'])/df_test['b4']

Aplicando a normalização

In [ ]:
data_train, data_test = normalize_dataset(df_train.drop(columns=['cent_price_cor','cent_trans_cor']), df_test.drop(columns=['id']))

Treinando com os dados de treino

In [ ]:
final_model = final_train(data_train, df_train[['cent_price_cor','cent_trans_cor']], epochs = 55)

Predizendo

In [ ]:
final_results = final_model.predict(data_test)

Transformando em DF

In [ ]:
final_predictions = pd.DataFrame(final_results, columns=['cent_price_cor', 'cent_trans_cor'])
final_predictions

In [ ]:
final_predictions.tail(20)

Persistindo em disco

In [ ]:
final_model.save('model.h5')

In [ ]:
final_predictions.to_csv('predictions.csv', index=0)

In [ ]:
the_model = tf.keras.models.load_model('model.h5')

In [ ]:
the_model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 28)                812       
_________________________________________________________________
dropout_19 (Dropout)         (None, 28)                0         
_________________________________________________________________
activation_19 (Activation)   (None, 28)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 2)                 58        
Total params: 870
Trainable params: 870
Non-trainable params: 0
_________________________________________________________________


A melhor combinação de todas as variáveis produziu um score de 0.184326

### 5.6 Another try

Em mais uma tentativa aplicamos o PCA para reduzir a dimensionalidade dos dados e mesclar features

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=10)
pca.fit(data_train)
x_train = pca.transform(data_train)
x_test = pca.transform(data_test)

In [ ]:
another_model = final_train(x_train, df_train[['cent_price_cor','cent_trans_cor']], epochs = 53)

In [ ]:
another_results = another_model.predict(x_test)

In [ ]:
another_predictions = pd.DataFrame(another_results, columns=['cent_price_cor', 'cent_trans_cor'])
another_predictions

In [ ]:
another_predictions.head(10)

In [ ]:
another_predictions.to_csv('predictions.csv', index=0)

In [ ]:
another_model.save('model.h5')

O melhor resultado com todas as combinações testadas nos retornou um score de 0.184321

## 6. Conclusão

Mesmo testando várias técnicas para minimizar o erro, o modelo conseguiu ter o melhor desempenho (0.18429) usando o modelo mais simples. Atingindo o segundo lugar ao término da primeira fase, contra 0.1812730 do primeiro colocado.